# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/data/'
CPG = 'coca' 
nielsen_cat = False #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = True
SOM_CPG = CPG.title()

## Import

In [2]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [3]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['nielsen_coca.csv', 'pasillos_coca.csv', 'products_coca.csv', 'som_coca.csv']


In [4]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16')
df.sample(2)

/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
34776,julio 2021,MX,Querétaro,Soriana,Milk,27,3154522,Leche sin lactosa light con proteína,{7501020550979},1 l,...,996,8,124,8,36,0,38,94.7%,94.7%,1.00
13724,julio 2021,MX,Ciudad de México,Sumesa,Plant-based Beverages,1516,338685,Bebida de almendra coco,{7502252481789},946 ml,...,38,1,38,1,1,0,1,100.0%,100.0%,1.00


In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,julio 2021,41317805.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    7501055349913
1    7501055307043
2    7501055357338
3          4936509
4    7501055349890
Name: UPC, dtype: int64

0    7501055349913
1    7501055307043
2    7501055357338
3          4936509
4    7501055349890
Name: UPC, dtype: int64

0    7501055349913
1    7501055307043
2    7501055357338
3          4936509
4    7501055349890
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
675,7501031310258,NaN,PepsiCo Beverages,NaN,NaN,Pepsi,2.5 Litros,Refresco de cola,343134,Beverages,NaN,NaN,NaN
1546,41220818631,NaN,Hill Country Fare,NaN,NaN,HCF,NaN,Bebida Sabor Citrus Punch 3.78 lt,203813,Fresh Juice & Nectar,NaN,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_1688/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


12553      36731143549
37805     813636021888
37972      16300165981
12895    7501295600614
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
16342,julio 2021,MX,Guadalajara,Chedraui,Hard Seltzer,1537,2797176,Hard seltzer sabor limón,7500464376992,Lata 355 ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37448,julio 2021,MX,Toluca,Chedraui,Water,35,1117374,Agua con toque sabor fresa,758104006663,1.5 l,...,Danone,nan,nan,Bonafont Levité,1.5 l,Agua con toque sabor fresa,Water,nan,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
julio 2021,41317805.00,41317805.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
110463,1,3726984,4062055079567,1312,Ropa deportiva mujer,245,Adidas,1211,Adidas
31768,1,219286,30111457868,230,Alimento seco,7189,Royal Canin,87,Royal Canin
7490,1,116560,7501035010314,1025,Mezclas de cóctel,774,José Cuervo,24,Casa Cuervo
43149,1,324209,31200458728,31,Jugos y néctares,549,Ocean Spray,1371,Ocean Spray


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
40528,1,315976,41220983230,90,Cereales,10001,HCF,1140,Hill Country Fare,Hill Country Fare (1140)
106359,1,3421526,7509546664217,964,Lociones y jabones,240,Palmolive,59,Colgate Palmolive,Colgate Palmolive (59)
2535,1,96241,7501005108096,29,Mantequilla,5678,Primavera,250,Upfield,Upfield (250)
72809,1,1106586,7506306209992,1500,Cuidado del cabello,128,Axe,44,Unilever,Unilever (44)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
julio 2021,41317805.00,41317805.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
4940,Chihuahua,Fresh Market,Fresh Juice & Nectar,31,312547,Jugo de kale espinaca apio pepino y nopal,7503021827050,1 L,nan,Frutos De Vida (4734),...,nan,nan,nan,KUA Foods (207),7,2021,4734,Frutos De Vida,207,KUA Foods
8404,Ciudad de México,City Market,Sparkling Water,1589,170413,Agua Mineral bui PEPINO 290 ml,7503015505209,nan,750301550520,bui (7437),...,nan,nan,nan,Bui Water (939),7,2021,7437,bui,939,Bui Water


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
jul,41317805.00,41317805.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
4634,Chihuahua,Fresh Market,Baby Food & Formula,53,335372,Jugo de pera etapa 2,7501000922246,175 ml,,Gerber (2482),...,Baby Food & Formula,,,,Nestlé MX (3),7,2021,2482,Gerber,3


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
28477,,Santa Clara MX,Santa Clara MX,Santa Clara,Santa Clara,250 ml,Leche sabor helado de menta con chocolate,Milk,,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
26367,,Nestlé Mx,Nestlé Mx,Gerber,Gerber,118 Ml,Agua Y Uva Roja,Baby Food & Formula,,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,jul,41317805.00


## Export

### Same product_id as past

In [22]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

38866
33394


In [23]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [24]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [25]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
16869,Querétaro,HEB,Beverages,36,344254,Refresco sabor lima limón,7501031305087,Botella 600 ml,,7 Up,...,7 Up,7 Up,Botella 600 Ml,Refresco Sabor Lima Limón,Beverages,,,,7,2021
22461,Guadalajara,City Market,Water,35,312472,Agua de matial,7501059200050,500 ml,,Sta. María,...,Sta. María,Sta. María,500 Ml,Agua De Matial,Water,,,,7,2021
22355,San Luis Potosí,Circle K,Water,35,312467,Agua de matial,7501059200012,1.5 l,,Sta. María,...,Sta. María,Sta. María,1.5 L,Agua De Matial,Water,,,,7,2021
15186,Monterrey,City Market,Baby Food & Formula,53,3324787,Cereal seco para lactantes de avena y arroz,7506475101172,270 g,446116,Gerber,...,Gerber,Gerber,270 G,Cereal Seco Para Lactantes De Avena Y Arroz,Baby Food & Formula,,,,7,2021


In [26]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [27]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
jul,41317805.00,36508619.00,-4809186.00,-0.12


In [28]:
time_exp(time.time() - start)
tono()

1 minutos con 2.46 segundos
